# Introduction to Creating Tables

To try out how to create tables and insert data into the tables we are using [duckdb](https://duckdb.org/) in this notebook. It is creating a local database which you can run SQL-queries in Jupyter Notebooks against. DBeaver also has a connector so you can also run SQL-queries in there. Duckdb works with sqlalchemy a widely used python library to connect to databases and ipython-sql. Ipython-sql allows us to convert a Jupyter code cell into a SQL cell with the jupyter magic command ```%%sql```.

In this notebook you will learn how to create tables and insert data into the created tables.

In [1]:

import duckdb
import pandas as pd
import sqlalchemy
import json
# No need to import duckdb_engine
# SQLAlchemy will auto-detect the driver needed bASed on your connection string!

# Import ipython-sql Jupyter extension to create SQL cells
%load_ext sql

Set configrations on ipython-sql to directly output data to Pandas and to simplify the output that is printed to the notebook.

In [2]:
%config SqlMagic.autopandAS = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

/Users/eikerogall/Documents/neuefische/test_duckdb/.venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3139: UserWarning: Config option `autopandAS` not recognized by `SqlMagic`.  Did you mean `autopandas`?
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Connect ipython-sql to DuckDB using a SQLAlchemy-style connection string. You may either connect to an in memory DuckDB, or a file backed db.

In [3]:
%sql duckdb:///sampleDB.duckdb


We have two csv files in our data directory:
- raw_orders.csv
- raw_payments.csv

and a json file:
- raw_customers.json


So we will see how to insert csv files and json files into our database but first we will create a tables for the data and for that let's have a look at the data and headers.

In [4]:
df_customers = pd.read_json('data/raw_customers.json', orient='records', lines=True)
df_customers.head()

,id,first_name,last_name
0,1,Michael,P.
1,2,Shawn,M.
2,3,Kathleen,P.
3,4,Jimmy,C.
4,5,Katherine,R.


In [5]:
df_orders = pd.read_csv('data/raw_orders.csv')
df_orders.head()

,id,user_id,order_date,status
0,1,1,2018-01-01,returned
1,2,3,2018-01-02,completed
2,3,94,2018-01-04,completed
3,4,50,2018-01-05,completed
4,5,64,2018-01-05,completed


In [6]:
df_payment = pd.read_csv('data/raw_payments.csv')
df_payment.head()

,id,order_id,payment_method,amount
0,1,1,credit_card,1000
1,2,2,credit_card,2000
2,3,3,coupon,100
3,4,4,coupon,2500
4,5,5,bank_transfer,1700


Let us start with creating the tables, for that it is standard to define the column types. The column types can differ from sql flavour to sql flavour.

We will start with the ```raw_cusomer``` table here we have:
- id
- first_name
- last_name

We will also define the ```id``` as ```INTEGER``` and ```PRIMARY KEY``` and additionally we say that it can't have missing values with ```NOT NULL```. The other two columns are strings which is call ```VARCHAR``` in sql.

In [7]:
%%sql

CREATE TABLE raw_customers(
    id INTEGER PRIMARY KEY NOT NULL,
    first_name VARCHAR,
    last_name VARCHAR
    );


Count


Next we will create the ```raw_orders``` table:
- id
- user_id
- order_date
- status

Again we will define the ```id``` as ```INTEGER``` and ```PRIMARY KEY``` and additionally we say that it can't have missing values with ```NOT NULL```. The ```user_id``` is a ```FOREIGN KEY``` and references the raw_customer id. Order_date is a ```DATE``` and status is a ```VARCHAR```.

In [8]:
%%sql

CREATE TABLE raw_orders(
    id INTEGER PRIMARY KEY NOT NULL,
    user_id INTEGER,
    FOREIGN KEY(user_id) REFERENCES raw_customers(id),
    order_date DATE,
    status VARCHAR
    );

Count


Last we will create the ```raw_payments``` table:
- id
- order_id
- payment_method
- amount

Again we will define the ```id``` as ```INTEGER``` and ```PRIMARY KEY``` and additionally we say that it can't have missing values with ```NOT NULL```. The ```order_id``` is a ```FOREIGN KEY``` and references the raw_orders id. payment_method is a ```VARCHAR``` and amount is a ```INTEGER```.

In [9]:
%%sql

CREATE TABLE raw_payments(
    id INTEGER PRIMARY KEY NOT NULL,
    order_id INTEGER,
    FOREIGN KEY(order_id) REFERENCES raw_orders(id),
    payment_method VARCHAR,
    amount INTEGER
    );

Count


So now we created the tables but at the moment they are empty. Let's check that.

In [10]:
%%sql

select * from raw_customers

id,first_name,last_name


In [11]:
%%sql

select * from raw_orders

id,user_id,order_date,status


In [12]:
%%sql

select * from raw_payments

id,order_id,payment_method,amount


As aspected they are empty, so let's change that and use ```INSERT``` for that. Most sql flavours have build in functions to deal with external data sources like csv or json files. In duckdb we can simply use ```read_csv_auto``` to read our csv files and we can than run sql queries against that. For json it is a little bit more complicated we have to create first a table where we store the json itself.

For json we we have to install and load the ```json``` extension from duckdb.

In [13]:
%%sql

INSTALL 'json';
LOAD 'json';

Success


Now we have to create the table we store the json in.

In [14]:
%%sql

CREATE TABLE raw_customers_json(
    j JSON
    );


Count


Now we can use the ```INSERT INTO``` command followed by a select statement where the json is read with ```read_csv_auto(<path>)```.

In [15]:
%%sql

INSERT INTO raw_customers_json SELECT * FROM read_json_objects('data/raw_customers.json')

Count
100


Let's have a look if that works:

In [16]:
%%sql

SELECT * FROM raw_customers_json

j
"{""id"":1,""first_name"":""Michael"",""last_name"":""P.""}"
"{""id"":2,""first_name"":""Shawn"",""last_name"":""M.""}"
"{""id"":3,""first_name"":""Kathleen"",""last_name"":""P.""}"
"{""id"":4,""first_name"":""Jimmy"",""last_name"":""C.""}"
"{""id"":5,""first_name"":""Katherine"",""last_name"":""R.""}"
"{""id"":6,""first_name"":""Sarah"",""last_name"":""R.""}"
"{""id"":7,""first_name"":""Martin"",""last_name"":""M.""}"
"{""id"":8,""first_name"":""Frank"",""last_name"":""R.""}"
"{""id"":9,""first_name"":""Jennifer"",""last_name"":""F.""}"
"{""id"":10,""first_name"":""Henry"",""last_name"":""W.""}"


Now that we have the json in a database we can parse it and put it into our ```raw_customer``` table, we can parse the json with the ```json_extract``` function with the json and the json_path as input e.g. ```json_extract(j, '$.id')```. We will also use the ```CAST``` function which converts a value (of any type) into a specified datatype. This will be included in a ```INSERT INTO``` and ```SELECT``` statement.

In [17]:
%%sql
INSERT INTO raw_customers
SELECT 
    CAST(json_extract(j, '$.id') AS INTEGER) AS id,
    REPLACE(CAST(json_extract(j, '$.first_name')AS VARCHAR),'"','') AS first_name, 
    REPLACE(CAST(json_extract(j, '$.last_name')AS VARCHAR),'"','') AS last_name 
FROM raw_customers_json;

Count
100


Inserting the data from the csv files is a little bit easier first for the orders:

In [19]:
%%sql

INSERT INTO raw_orders SELECT id,user_id,order_date,status FROM read_csv_auto('data/raw_orders.csv');

Count
99


And for the payments:

In [20]:
%%sql

INSERT INTO raw_payments SELECT id,order_id,payment_method,amount FROM read_csv_auto('data/raw_payments.csv');

Count
113


Now let's have a look into the data with SQL:

In [21]:
%%sql

SELECT * FROM raw_customers

id,first_name,last_name
1,Michael,P.
2,Shawn,M.
3,Kathleen,P.
4,Jimmy,C.
5,Katherine,R.
6,Sarah,R.
7,Martin,M.
8,Frank,R.
9,Jennifer,F.
10,Henry,W.


In [22]:
%%sql

SELECT * FROM raw_orders

id,user_id,order_date,status
1,1,2018-01-01,returned
2,3,2018-01-02,completed
3,94,2018-01-04,completed
4,50,2018-01-05,completed
5,64,2018-01-05,completed
6,54,2018-01-07,completed
7,88,2018-01-09,completed
8,2,2018-01-11,returned
9,53,2018-01-12,completed
10,7,2018-01-14,completed


In [23]:
%%sql

SELECT * FROM raw_payments

id,order_id,payment_method,amount
1,1,credit_card,1000
2,2,credit_card,2000
3,3,coupon,100
4,4,coupon,2500
5,5,bank_transfer,1700
6,6,credit_card,600
7,7,credit_card,1600
8,8,credit_card,2300
9,9,gift_card,2300
10,9,bank_transfer,0
